# Fuzzy Name Matching Exercise

The goal of this project is to identify how many companies in 'List A' are also present in 'List B'. 

We'll use fuzzy matching, meaning we'll do string comparisons and get a score, and then make a judgement call on whether we consider it a match.

Desired output: 

| ListA_original | ListA_cleaned | ListB_original | ListB_cleaned | Score |
| --- | --- | --- | --- | --- |
| Microsoft Corp. | microsoft | MicroSoft | microsoft | 100 |

### Step 1: Import packages and initial datasets

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [83]:
list1 = ['Walmart', 'Amazon', 'Apple', 'Facebook', 'Microsoft', 'Company 23', 'onmicrosoft', 'testcompany', 'trialco']
list2 = ['Wal-mart', 'Amazon Co.', 'Apple Inc.', 'Facebook', 'Microsoft Corp.', 'company23inc']

### Step 2: Data Cleaning
Approach: 
- List of company names, get count
- Deduplicate company names, get count
- Clean list of company names, get count
- Deduplicate list of cleaned company names, get count; use final list for fuzzy matching

In [40]:
# create a list of strings we will use to remove from the company names
list_remove = [' co.', ' inc.', ' corp.']

In [61]:
# create a function that takes a list of strings, a list of strings to remove, and a string 'label' as input
# and returns a dataframe that has two columns called 'original' and 'cleaned'
# where 'cleaned' is the string in the list that is converted to lowercase, has punctions removed, and has the strings in the list of strings to remove removed
# and 'original' is the original string in the list
def gen_df_clean(inpt, remove, label):
    df = pd.DataFrame(inpt, columns = ['original'])
    df['cleaned'] = df['original'].str.lower().str.replace('[^\w\s]','')
    # remove the list_remove strings from strings in list inpt
    for i in range(len(list_remove)):
        df['cleaned'] = [x.replace(list_remove[i], '') for x in df['cleaned']]
    # remove all spaces and characters that are not a-0, A-Z, or 0-9
    df['cleaned'] = df['cleaned'].str.replace(' ','').str.replace('[^a-zA-Z0-9]','')
    # modify the column names to include the label
    df.columns = [label + '_original', label + '_cleaned']
    return df



In [84]:
df_pitchbook = gen_df_clean(list1, list_remove, 'pitchbook')
df_crunchbase = gen_df_clean(list2, list_remove, 'crunchbase')
# create a dataframe with only the second column
ListA_Cleaned = df_pitchbook[['pitchbook_cleaned']]
# rename column "ListA" 
ListA_Cleaned = ListA_Cleaned.rename(columns={'pitchbook_cleaned': 'ListA'})

ListB_Cleaned = df_crunchbase[['crunchbase_cleaned']]
# rename column "ListB"
ListB_Cleaned = ListB_Cleaned.rename(columns={'crunchbase_cleaned': 'ListB'})

print(ListA_Cleaned)
print(ListB_Cleaned)

         ListA
0      walmart
1       amazon
2        apple
3     facebook
4    microsoft
5    company23
6  onmicrosoft
7  testcompany
8      trialco
          ListB
0      wal-mart
1        amazon
2         apple
3      facebook
4     microsoft
5  company23inc


In [87]:
def gen_fuzzy_score (df1, df2): 
    df_score = pd.DataFrame(columns = ['ListA', 'ListB', 'Score'])
    for i in range(len(ListA_Cleaned)):
        i_score = ['a', 'b', 0]
        for j in range(len(ListB_Cleaned)):
            # iterate i through each string in ListB_Cleaned
            # if score = 100, then the strings are identical and we can stop iterating and replace i_score with i, j, and the score
            # else, continue iterating and if the score is higher than the previous score, replace i_score with i, j, and the score
            score = fuzz.ratio(ListA_Cleaned.iloc[i,0], ListB_Cleaned.iloc[j,0])
            if score == 100:
                i_score = [ListA_Cleaned.iloc[i,0], ListB_Cleaned.iloc[j,0], score]
                break
            elif score > i_score[2]:
                i_score = [ListA_Cleaned.iloc[i,0], ListB_Cleaned.iloc[j,0], score]
        # append the i_score to the dataframe using concat
        df_score = pd.concat([df_score, pd.DataFrame([i_score], columns = ['ListA', 'ListB', 'Score'])], ignore_index=True)
    return df_score


In [88]:
output = gen_fuzzy_score(ListA_Cleaned, ListB_Cleaned)
output

,ListA,ListB,Score
0,walmart,wal-mart,93
1,amazon,amazon,100
2,apple,apple,100
3,facebook,facebook,100
4,microsoft,microsoft,100
5,company23,company23inc,86
6,onmicrosoft,microsoft,90
7,testcompany,company23inc,61
8,trialco,facebook,40


### Step 3: Fuzzy Match Scoring
Approach: 
- Take two lists of cleaned and deduplicated company names and create a new list that has the top matched score (even if it's a low score)

In [58]:
# create a function that takes two dataframes as inputs
# and populates the empty dataframe with the original and cleaned strings from each dataframe and the fuzzywuzzy score
# only keep the top score per row in df1
def gen_df_fuzzy(df1, df2):
    df = pd.DataFrame(columns = ['List1_original', 'List1_cleaned', 'List2_original', 'List2_cleaned', 'Score'])
    for i in range(len(df1)):
        for j in range(len(df2)):
            score = fuzz.ratio(df1.iloc[i,1], df2.iloc[j,1])
            # if score is greater than the score in the dataframe, replace the score and the other columns
            if score > df['Score'].iloc[i]:
                df.loc[i] = [df1.iloc[i,0], df1.iloc[i,1], df2.iloc[j,0], df2.iloc[j,1], score]
    return df


In [64]:
# create a function that takes two dataframes as inputs
# compare the second column in the first dataframe to the second column in the second dataframe
# only keep the top score per row in df1
def gen_df_fuzzy2(df1, df2):
    df = pd.DataFrame(columns = ['List1_original', 'List1_cleaned', 'List2_original', 'List2_cleaned', 'Score'])
    for i in range(len(df1)):
        for j in range(len(df2)):
            score = fuzz.ratio(df1.iloc[i,1], df2.iloc[j,1])
            # if score is greater than the score in the dataframe, replace the score and the other columns
            if score > df['Score'].iloc[i]:
                df.loc[i] = [df1.iloc[i,0], df1.iloc[i,1], df2.iloc[j,0], df2.iloc[j,1], score]
    return df

In [65]:
df_pitchbook = gen_df_clean(list1, list_remove, 'pitchbook')
df_crunchbase = gen_df_clean(list2, list_remove, 'crunchbase')

df_score = gen_df_fuzzy2(df_pitchbook, df_crunchbase)

IndexError: single positional indexer is out-of-bounds

In [63]:
df_score

,List1_original,List1_cleaned,List2_original,List2_cleaned,Score
0,Walmart,walmart,Wal-mart,wal-mart,93
1,Walmart,walmart,Amazon Co.,amazon,46
2,Walmart,walmart,Apple Inc.,apple,33
3,Walmart,walmart,Facebook,facebook,13
4,Walmart,walmart,Microsoft Corp.,microsoft,38
5,Walmart,walmart,company23inc,company23inc,21
6,Amazon,amazon,Wal-mart,wal-mart,43
7,Amazon,amazon,Amazon Co.,amazon,100
8,Amazon,amazon,Apple Inc.,apple,18
9,Amazon,amazon,Facebook,facebook,29


In [14]:
# create an empty dataframe with column names 'List1', 'List2', 'Score'
df_score = pd.DataFrame(columns=['List1', 'List2', 'Score'])

# iterate over each element in list1 and compare with each element in list2
for i in list1:
    for j in list2:
        # calculate the score for each pair of strings
        score = fuzz.ratio(i, j)
        # create a dataframe with the values
        df = pd.DataFrame([[i, j, score]], columns=['List1', 'List2', 'Score'])
        # append the dataframe to df_score using concat()
        df_score = pd.concat([df_score, df], ignore_index=True)

# print the final dataframe
print(df_score)

        List1      List2 Score
0     walmart    walmart   100
1     walmart     amazon    46
2     walmart      apple    33
3     walmart   facebook    13
4     walmart  microsoft    38
5      amazon    walmart    46
6      amazon     amazon   100
7      amazon      apple    18
8      amazon   facebook    29
9      amazon  microsoft    27
10      apple    walmart    33
11      apple     amazon    18
12      apple      apple   100
13      apple   facebook    31
14      apple  microsoft     0
15   facebook    walmart    13
16   facebook     amazon    29
17   facebook      apple    31
18   facebook   facebook   100
19   facebook  microsoft    35
20  microsoft    walmart    38
21  microsoft     amazon    27
22  microsoft      apple     0
23  microsoft   facebook    35
24  microsoft  microsoft   100


In [15]:
# create a new column 'Rank' which ranks the scores in descending order per item in List1
df_score['Rank'] = df_score.groupby('List1')['Score'].rank(method='first', ascending=False)

# print the final dataframe
print(df_score)

        List1      List2 Score  Rank
0     walmart    walmart   100   1.0
1     walmart     amazon    46   2.0
2     walmart      apple    33   4.0
3     walmart   facebook    13   5.0
4     walmart  microsoft    38   3.0
5      amazon    walmart    46   2.0
6      amazon     amazon   100   1.0
7      amazon      apple    18   5.0
8      amazon   facebook    29   3.0
9      amazon  microsoft    27   4.0
10      apple    walmart    33   2.0
11      apple     amazon    18   4.0
12      apple      apple   100   1.0
13      apple   facebook    31   3.0
14      apple  microsoft     0   5.0
15   facebook    walmart    13   5.0
16   facebook     amazon    29   4.0
17   facebook      apple    31   3.0
18   facebook   facebook   100   1.0
19   facebook  microsoft    35   2.0
20  microsoft    walmart    38   2.0
21  microsoft     amazon    27   4.0
22  microsoft      apple     0   5.0
23  microsoft   facebook    35   3.0
24  microsoft  microsoft   100   1.0


In [16]:
# create a new dataframe with the top ranked scores per item in List1
df_top = df_score[df_score['Rank'] == 1]

# print the final dataframe
print(df_top)

        List1      List2 Score  Rank
0     walmart    walmart   100   1.0
6      amazon     amazon   100   1.0
12      apple      apple   100   1.0
18   facebook   facebook   100   1.0
24  microsoft  microsoft   100   1.0
